In [0]:
# libraries installation for serverless compute
%pip install geopy timezonefinder

# Cricinfo Commentary Parser - Silver Layer (Incremental Load via Delta CDF)

**Bronze Layer Tables (CDF Enabled):**
- `T20_catalog.bronze.match_events` (ball-by-ball CSV)
- `T20_catalog.bronze.match_metadata` (match-level JSON)
- `T20_catalog.bronze.match_players` (player-level CSV)

## Setup & Configuration

In [0]:
# ── Job Parameters ────────────────────────────────────────────────────────────
# Default values are used during interactive runs.
# Databricks Job overrides these at runtime via the Parameters section.
# Key names here must match exactly what the Job defines.

dbutils.widgets.text(
    "catalog_name", "T20_catalog",
    "Catalog Name"
)


In [0]:
from pyspark.sql.functions import (
    col, when, regexp_extract, regexp_replace, trim, first, create_map,
    coalesce, collect_set, lit, current_timestamp, split, size,
    sum as spark_sum, udf, to_timestamp, concat, to_utc_timestamp,
    to_date, try_element_at, monotonically_increasing_id, row_number,element_at,
    desc, asc
)
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from pyspark.sql import Row
from itertools import chain
from functools import reduce
import json
import time

# Unity Catalog Configuration
CATALOG  = dbutils.widgets.get("catalog_name")
BRONZE = f"{CATALOG}.bronze"
SILVER = f"{CATALOG}.silver"

# Table names
BRONZE_EVENTS   = f"{BRONZE}.match_events"
BRONZE_PLAYERS  = f"{BRONZE}.match_players"
BRONZE_METADATA = f"{BRONZE}.match_metadata"
SILVER_EVENTS   = f"{SILVER}.match_events"
SILVER_PLAYERS  = f"{SILVER}.match_players"
SILVER_METADATA = f"{SILVER}.match_metadata"
TIMEZONE_CACHE_TABLE = f"{SILVER}.ground_timezone_map"
CDF_CHECKPOINT_TABLE = f"{SILVER}.cdf_checkpoint"

# CDF metadata columns to drop after filtering
CDF_META_COLS = ["_change_type", "_commit_version", "_commit_timestamp"]

## Create Checkpoint & Silver Schema (One-time Setup)

In [0]:
# Create silver schema if not exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SILVER}")

# Checkpoint table tracks the last processed Delta version per source table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {CDF_CHECKPOINT_TABLE} (
        source_table STRING,
        last_processed_version LONG,
        last_processed_timestamp TIMESTAMP
    )
    USING DELTA
""")

## CDF Helper Functions & Utility Functions

| Function | Purpose |
|----------|---------|
| `read_cdf_changes()` | Read CDF from _delta_log since last checkpoint |
| `get_affected_match_ids()` | Extract matchids from CDF insert/update rows |
| `dedup_dataframe()` | Drop duplicates excluding metadata columns |
| `dedup_column_values()` | Remove repetitive text within column values |
| `merge_to_silver()` | Unified merge/create logic for any Silver table |
| `propagate_deletes()` | Unified CDF delete propagation |
| `build_ground_timezone_map()` | Geocode grounds → timezone with Delta cache |
| `local_time_to_utc()` | Convert local match times to UTC |

In [0]:
# ============================================================
# CDF Functions
# ============================================================

def get_last_processed_version(table_name):
    """Get the last successfully processed CDF version for a source table."""
    try:
        result = spark.sql(f"""
            SELECT last_processed_version 
            FROM {CDF_CHECKPOINT_TABLE}
            WHERE source_table = '{table_name}'
        """).collect()
        return result[0][0] if result else None
    except Exception:
        return None


def get_current_table_version(table_name):
    """Get the latest Delta version of a table."""
    return spark.sql(f"DESCRIBE HISTORY {table_name} LIMIT 1").collect()[0]["version"]


def get_cdf_enabled_version(table_name):
    """Find the Delta version when CDF was first enabled."""
    history = spark.sql(f"DESCRIBE HISTORY {table_name}").collect()
    for row in reversed(history):
        ops = row["operationParameters"] or {}
        if "delta.enableChangeDataFeed" in str(ops):
            return row["version"]
    return get_current_table_version(table_name)


def update_checkpoint(table_name, version):
    """Update the CDF checkpoint with the latest processed version."""
    spark.sql(f"""
        MERGE INTO {CDF_CHECKPOINT_TABLE} AS target
        USING (
            SELECT '{table_name}' AS source_table,
                   {version} AS last_processed_version,
                   current_timestamp() AS last_processed_timestamp
        ) AS source
        ON target.source_table = source.source_table
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)


def read_cdf_changes(table_name):
    """Read file-level changes from the Delta transaction log since last checkpoint."""
    current_version = get_current_table_version(table_name)
    #print(current_version)
    silver_exists = spark.catalog.tableExists(SILVER_EVENTS)
    #print(silver_exists)
    if not silver_exists:
        print(f"  📥 {table_name}: First run → full table read")
        return spark.read.table(table_name), current_version

    last_version = get_last_processed_version(table_name)

    if last_version is not None and last_version >= current_version:
        print(f"  ⏭️  {table_name}: No new commits (at version {current_version})")
        return None, current_version

    start_version = (last_version + 1) if last_version is not None else get_cdf_enabled_version(table_name)

    if start_version > current_version:
        print(f"  ⏭️  {table_name}: No CDF changes after version {start_version}")
        return None, current_version

    print(f"  📥 {table_name}: CDF version {start_version} → {current_version}")
    return (
        spark.read.format("delta")
        .option("readChangeFeed", "true")
        .option("startingVersion", start_version)
        .option("endingVersion", current_version)
        .table(table_name)
    ), current_version


def get_affected_match_ids(df_cdf):
    """Extract distinct matchid values from CDF or full-read DataFrames."""
    if df_cdf is None:
        return set()
    
    # First run: full table read has no _change_type column
    if "_change_type" in df_cdf.columns:
        df_filtered = df_cdf.filter(col("_change_type").isin("insert", "update_postimage"))
    else:
        df_filtered = df_cdf  # Full read — all rows are "new"
    
    return set(
        row.matchid for row in
        df_filtered.select("matchid").distinct().collect()
    )


# ============================================================
# Merge & Delete Helpers (eliminates repeated code)
# ============================================================

def merge_to_silver(df_source, silver_table_name, merge_keys):
    """Unified merge/create for any Silver table.
    
    Args:
        df_source: DataFrame to merge
        silver_table_name: Full Silver table name
        merge_keys: List of columns for merge condition
    """
    table_exists = spark.catalog.tableExists(silver_table_name)

    if not table_exists:
        print(f"  🆕 Creating {silver_table_name} (first run)...")
        (
            df_source.write.format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .saveAsTable(silver_table_name)
        )
    else:
        print(f"  🔄 Merging into {silver_table_name}...")
        condition = " AND ".join([f"target.{k} = source.{k}" for k in merge_keys])
        (
            DeltaTable.forName(spark, silver_table_name).alias("target")
            .merge(df_source.alias("source"), condition)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
    print(f"  ✅ {silver_table_name} complete")
    return table_exists


def propagate_deletes(df_cdf, silver_table_name, merge_keys, table_existed):
    """Unified CDF delete propagation for any Silver table."""
    if df_cdf is None or not table_existed:
        return
    df_deletes = df_cdf.filter(col("_change_type") == "delete")
    # Cache and count once
    #df_deletes.cache()
    df_deletes.createOrReplaceTempView("deletes_temp")
    df_deletes = spark.table("deletes_temp")

    delete_count = df_deletes.count()
    if delete_count > 0:
        print(f"  🗑️  Propagating {delete_count} deletes to {silver_table_name}...")
        condition = " AND ".join([f"target.{k} = source.{k}" for k in merge_keys])
        (
            DeltaTable.forName(spark, silver_table_name).alias("target")
            .merge(df_deletes.alias("source"), condition)
            .whenMatchedDelete()
            .execute()
        )
        print(f"  ✅ {delete_count} records deleted")
    else:
        print(f"  🗑️  No CDF deletes for {silver_table_name}")
    #df_deletes.unpersist()


# ============================================================
# Data Quality / Dedup Functions
# ============================================================

@udf(StringType())
def remove_duplicate_text(value):
    """Remove duplicate/repetitive entries from a comma-delimited string."""
    if not value:
        return value
    parts = [p.strip() for p in value.split(" ")]  # FIX: split by comma, not space
    seen = set()
    unique = []
    for part in parts:
        if part and part.lower() not in seen:
            seen.add(part.lower())
            unique.append(part)
    return " ".join(unique)


def dedup_column_values(df, columns):
    """Remove duplicate/repetitive text from specified columns."""
    valid_cols = [c for c in columns if c in df.columns]
    skipped_cols = [c for c in columns if c not in df.columns]
    for c in valid_cols:
        df = df.withColumn(c, remove_duplicate_text(trim(col(c))))
    print(f"  🧹 Deduped text in {len(valid_cols)} column(s): {', '.join(valid_cols)}")
    if skipped_cols:
        print(f"     Skipped (not found): {', '.join(skipped_cols)}")
    return df


def dedup_dataframe(df, exclude_cols=None, order_by_col=None, keep="latest"):
    """Drop duplicates excluding metadata columns. Uses window-based dedup."""
    if exclude_cols is None:
        exclude_cols = ["load_timestamp", "batch_id", "source_file", "_rescued_data"]

    exclude_cols = [c for c in exclude_cols if c in df.columns]
    dedup_cols = [c for c in df.columns if c not in exclude_cols]

    if order_by_col and order_by_col in df.columns:
        sort_order = desc(order_by_col) if keep == "latest" else asc(order_by_col)
        window = Window.partitionBy(dedup_cols).orderBy(sort_order)
        df_result = (
            df.withColumn("_rank", row_number().over(window))
            .filter(col("_rank") == 1)
            .drop("_rank")
        )
    else:
        df_result = df.dropDuplicates(dedup_cols)

    print(f"  🔄 Dedup complete on {len(dedup_cols)} columns")
    return df_result


# ============================================================
# Timezone Functions
# ============================================================

def build_ground_timezone_map(df, ground_col="ground"):
    """Geocode grounds → timezone with Delta cache + city fallback."""
    cached_map = {}
    cache_exists = spark.catalog.tableExists(TIMEZONE_CACHE_TABLE)

    if cache_exists:
        cached_map = {r.ground: r.local_timezone for r in spark.read.table(TIMEZONE_CACHE_TABLE).collect()}
        print(f"  📦 Cache: {len(cached_map)} ground(s)")

    all_grounds = [
        r[ground_col] for r in
        df.select(ground_col).distinct().filter(col(ground_col).isNotNull()).collect()
    ]
    new_grounds = [g for g in all_grounds if g not in cached_map]

    if not new_grounds:
        print(f"  ✅ All {len(all_grounds)} ground(s) cached")
        return cached_map

    print(f"  🌍 Geocoding {len(new_grounds)} new ground(s)...")
    geolocator = Nominatim(user_agent="cricket_data_pipeline", timeout=10)
    tf = TimezoneFinder()
    new_entries = {}

    def _geocode(query):
        try:
            loc = geolocator.geocode(query)
            time.sleep(1.1)
            if loc:
                return tf.timezone_at(lat=loc.latitude, lng=loc.longitude)
        except Exception:
            pass
        return None

    for ground in new_grounds:
        short = ground.split(",")[0].strip()
        city_parts = [p.strip() for p in ground.split(",")]
        city = city_parts[-1] if len(city_parts) >= 2 else None

        # Try: full name → short + cricket stadium → city
        tz = _geocode(ground) or _geocode(short + " cricket stadium") or (city and _geocode(city))

        new_entries[ground] = tz
        status = f"✅ {tz}" if tz else "❌ unresolved"
        print(f"     {ground} → {status}")

    # Update cache
    valid_new = {k: v for k, v in new_entries.items() if v}
    if valid_new:
        df_new = spark.createDataFrame([Row(ground=k, local_timezone=v) for k, v in valid_new.items()])
        if cache_exists:
            DeltaTable.forName(spark, TIMEZONE_CACHE_TABLE).alias("t").merge(
                df_new.alias("s"), "t.ground = s.ground"
            ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
        else:
            # ✗ BEFORE — saveAsTable fails on serverless
            # df_new.write.format("delta").mode("overwrite").saveAsTable(TIMEZONE_CACHE_TABLE)

            # ✓ AFTER
            df_new.createOrReplaceTempView("_tz_cache_temp")
            spark.sql(f"""
                CREATE OR REPLACE TABLE {TIMEZONE_CACHE_TABLE}
                USING DELTA
                AS SELECT * FROM _tz_cache_temp
                """)
        print(f"  💾 Cache updated: +{len(valid_new)} ground(s)")

    unresolved = [k for k, v in new_entries.items() if not v]
    if unresolved:
        print(f"  ⚠️  Unresolved: {unresolved}")

    return {**cached_map, **valid_new}


def apply_timezone_map(df, timezone_map, ground_col="ground", output_col="local_timezone"):
    """Apply ground → timezone mapping to DataFrame — serverless compatible."""
    valid_map = {k: v for k, v in timezone_map.items() if v}
    mapping_expr = create_map([lit(x) for x in chain(*valid_map.items())])
    
    # ✗ BEFORE — bracket notation fails on serverless
    # return df.withColumn(output_col, mapping_expr[col(ground_col)])
    
    # ✓ AFTER — element_at works everywhere
    return df.withColumn(output_col, element_at(mapping_expr, col(ground_col)))

def local_time_to_utc(df, time_col, date_col, tz_col, output_col):
    """Convert HH.MM local time to UTC timestamp."""
    return df.withColumn(output_col,
        to_utc_timestamp(
            to_timestamp(
                concat(col(date_col).cast("string"), lit(" "),
                       regexp_replace(col(time_col), r"\.", ":")),
                "yyyy-MM-dd HH:mm"
            ), col(tz_col)
        )
    )

## Name Resolution UDF

In [0]:
# Known team abbreviations that don't follow standard patterns
KNOWN_TEAM_ABBREVS = {
    "USA": "United States Of America", "NED": "Netherlands", "BHR": "Bahrain",
    "NZ": "New Zealand", "WI": "West Indies", "SL": "Sri Lanka",
    "PNG": "Papua New Guinea", "UAE": "United Arab Emirates", "HK": "Hong Kong",
    "RSA": "South Africa", "SA": "South Africa", "ENG": "England",
    "AUS": "Australia", "IND": "India", "PAK": "Pakistan",
    "BAN": "Bangladesh", "AFG": "Afghanistan", "ZIM": "Zimbabwe",
    "IRE": "Ireland", "SCO": "Scotland", "NAM": "Namibia",
    "NEP": "Nepal", "OMA": "Oman", "CAN": "Canada", "UGA": "Uganda",
}


@udf(StringType())
def get_full_name(short_name, name_list):
    """Resolve abbreviated team/player name to full name using fuzzy matching."""
    if not short_name or not name_list:
        return short_name

    # Parse name_list
    if isinstance(name_list, list):
        names = name_list
    elif isinstance(name_list, str):
        try:
            names = json.loads(name_list)
        except (json.JSONDecodeError, ValueError):
            names = [n.strip().strip('"') for n in name_list.strip("[]").split(",")]
    else:
        return short_name

    if short_name in names:
        return short_name
    if "Super Over" in short_name:
        return short_name

    abbr = short_name.strip()
    abbr_upper = abbr.upper()
    abbr_parts = abbr.split()

    # 1. Known abbreviations lookup
    if abbr_upper in KNOWN_TEAM_ABBREVS:
        known_full = KNOWN_TEAM_ABBREVS[abbr_upper]
        for name in names:
            if name.upper() == known_full.upper():
                return name

    for name in names:
        name_parts = name.split()
        name_upper = name.upper()

        # 2. All-word initials (SA -> South Africa)
        initials = "".join(w[0] for w in name_parts).upper()
        if abbr_upper == initials:
            return name

        # 3. Initials skipping lowercase words (USA -> United States Of America)
        initials_upper_only = "".join(w[0] for w in name_parts if w[0].isupper()).upper()
        if abbr_upper == initials_upper_only and initials_upper_only != initials:
            return name

        # 4. Starts-with (AFG -> Afghanistan)
        if len(abbr) >= 2 and len(abbr_parts) == 1 and name_upper.startswith(abbr_upper):
            return name

        # 5. Exact last name (Stubbs -> Tristan Stubbs)
        if len(abbr_parts) == 1 and abbr_upper == name_parts[-1].upper():
            return name

        # 6. Multi-word surname tail (de Kock -> Quinton de Kock)
        if len(abbr_parts) >= 2 and len(name_parts) > len(abbr_parts):
            tail = " ".join(name_parts[-len(abbr_parts):]).upper()
            if abbr_upper == tail:
                return name

        # 7. Concatenated initials + surname (JN Loftie-Eaton -> Jan Nicol Loftie-Eaton)
        if len(abbr_parts) >= 2:
            first_part = abbr_parts[0].upper().rstrip(".")
            abbr_surname = " ".join(abbr_parts[1:]).upper()
            if (first_part.isalpha()
                and first_part.isupper()
                and len(first_part) >= 2
                and len(first_part) <= len(name_parts) - 1):
                name_initials = "".join(name_parts[i][0].upper() for i in range(len(first_part)))
                remaining = " ".join(name_parts[len(first_part):]).upper()
                if first_part == name_initials and abbr_surname == remaining:
                    return name

        # 8. Single initial + last name (M Nabi -> Mohammad Nabi)
        if len(abbr_parts) >= 2 and len(name_parts) >= 2:
            abbr_first = abbr_parts[0].upper().rstrip(".")
            if (len(abbr_first) == 1
                and abbr_first == name_parts[0][0].upper()
                and abbr_parts[-1].upper() == name_parts[-1].upper()):
                return name

        # 9. Abbreviated first + full last (R Khan -> Rashid Khan)
        if len(abbr_parts) >= 2 and len(name_parts) >= 2:
            abbr_last = " ".join(abbr_parts[1:]).upper()
            name_last = " ".join(name_parts[1:]).upper()
            abbr_f = abbr_parts[0].upper().rstrip(".")
            if abbr_last == name_last and name_parts[0].upper().startswith(abbr_f):
                return name

        # 10. Partial first + matching last name
        if len(abbr_parts) >= 2 and len(name_parts) >= 2:
            if (abbr_parts[-1].upper() == name_parts[-1].upper()
                and name_parts[0].upper().startswith(abbr_parts[0].upper())):
                return name

    return short_name

## Read CDF Changes from Bronze Tables

In [0]:
print("Reading Change Data Feed from Bronze tables...\n")

df_events_cdf, events_version     = read_cdf_changes(BRONZE_EVENTS)
df_players_cdf, players_version   = read_cdf_changes(BRONZE_PLAYERS)
df_metadata_cdf, metadata_version = read_cdf_changes(BRONZE_METADATA)
#df_events_cdf.display()
#print(events_version)

if df_events_cdf is None and df_players_cdf is None and df_metadata_cdf is None:
    print("\n✅ No new commits in Bronze. Silver layer is up to date.")
    dbutils.notebook.exit("NO_CHANGES")

## Collect Affected Match IDs

In [0]:
event_match_ids    = get_affected_match_ids(df_events_cdf)
player_match_ids   = get_affected_match_ids(df_players_cdf)
metadata_match_ids = get_affected_match_ids(df_metadata_cdf)

affected_match_ids = list(event_match_ids | player_match_ids | metadata_match_ids)
print(affected_match_ids)

print(f"📊 Affected matches: Events={len(event_match_ids)}, "
      f"Players={len(player_match_ids)}, Metadata={len(metadata_match_ids)}, "
      f"Total={len(affected_match_ids)}")

## Load Data for Affected Matches

In [0]:
# Events: CDF if available, else full read for player/metadata-only changes
if df_events_cdf is not None:
    if "_change_type" in df_events_cdf.columns:
        df_events_to_process = (
            df_events_cdf
            .filter(col("_change_type").isin("insert", "update_postimage"))
            .drop(*CDF_META_COLS)
        )
    else:
        # First run: full read, no CDF columns to drop
        df_events_to_process = df_events_cdf
else:
    df_events_to_process = (
        spark.read.table(BRONZE_EVENTS)
        .filter(col("matchid").isin(affected_match_ids))
    )

# Players & Metadata: always full read for affected matches
df_players_full = spark.read.table(BRONZE_PLAYERS).filter(col("matchid").isin(affected_match_ids))
df_metadata_full = spark.read.table(BRONZE_METADATA).filter(col("matchid").isin(affected_match_ids))
# Build player name lookup once (reused by events & metadata)
df_match_names = (
    df_players_full.groupBy("matchid")
    .agg(
        collect_set("team").alias("team_names"),
        collect_set("player_name").alias("player_names")
    )
)
df_match_names.createOrReplaceTempView("match_names_temp")
df_match_names = spark.table("match_names_temp")

print(f"📋 Loaded: Events={df_events_to_process.count()}, "
      f"Players={df_players_full.count()}, Metadata={df_metadata_full.count()}")

# Applying Transformations on Events Table

### Parse Event Column, Calculate Runs, Extract Overs

In [0]:
# Dedup events
df_events = dedup_dataframe(df_events_to_process, order_by_col="load_timestamp", keep="latest")

# Parse: "Bowler to Batsman, Score"
EVENT_PATTERN = r"([A-Za-z\s\'\-]*)\sto\s([A-Za-z\s\'\-]*)\,\s+([0-9A-Za-z\,\s\(\)]*)"

df_events = (
    df_events
    .withColumn("Bowler", trim(regexp_extract("event", EVENT_PATTERN, 1)))
    .withColumn("Batsman", trim(regexp_extract("event", EVENT_PATTERN, 2)))
    .withColumn("runs_text", trim(regexp_replace(
        trim(regexp_extract("event", EVENT_PATTERN, 3)), r"\(no ball\)", "")))
    .withColumn("Extras",
        when(col("runs_text").contains("wide"), "wide")
        .when(col("event").contains("no ball"), "noball")
        .when(col("runs_text").contains("leg bye"), "legbyes")
        .when(col("runs_text").contains("bye"), "byes")
        .otherwise("normal"))
    .withColumn("over", split(col("ball").cast("string"), "\\.")[0].cast("int") + 1)
    .withColumn("over_ball_number", split(col("ball").cast("string"), "\\.")[1].cast("int"))
    .withColumn("runs",
        when(col("runs_text") == "OUT", 0)
        .when((col("runs_text") == "no run") | (col("runs_text") == ""), 0)
        .when(col("runs_text") == "FOUR runs", 4)
        .when(col("runs_text") == "SIX runs", 6)
        .when(col("runs_text").rlike("^\\d+"),
              regexp_extract("runs_text", r"^(\d+)", 1).cast("int"))
        .otherwise(0))
    # Add no-ball penalty
    .withColumn("runs",
        when(col("Extras") == "noball", col("runs") + 1).otherwise(col("runs")))
)

### Extract Dismissal Information

In [0]:
is_out = col("runs_text").contains("OUT")
comm = col("commentary")

PAT_RUNOUT  = r"run out\s*\("
PAT_CANDB   = r"c & b\s+"
PAT_CAUGHT  = r"\bc\s+.+?\s+b\s+.+?\s+\d+\s*\("
PAT_LBW     = r"lbw\s+b\s+"
PAT_STUMPED = r"(?:^|[^a-zA-Z])st\s+[^a-zA-Z]*[A-Z].+?\s+b\s+.+?\s+\d+\s*\("
PAT_BOWLED  = r"(?:^|[^a-zA-Z])b\s+\w+\s+\d+\s*\("

df_events = (
    df_events
    .withColumn("dismissal_method",
        when(~is_out, "Not Out")
        .when(comm.rlike(PAT_RUNOUT), "Run Out")
        .when(comm.rlike(PAT_CANDB), "Caught & Bowled")
        .when(comm.rlike(PAT_CAUGHT), "Caught")
        .when(comm.rlike(PAT_LBW), "LBW")
        .when(comm.rlike(PAT_STUMPED), "Stumped")
        .when(comm.rlike(PAT_BOWLED), "Bowled")
        .otherwise("Not Out"))
    .withColumn("dismissed_by",
        when(~is_out, lit(None))
        .when(comm.rlike(PAT_RUNOUT),
             regexp_extract("commentary", r"run out\s*\(([^/\)]+)", 1))
        .when(comm.rlike(PAT_CANDB),
             regexp_extract("commentary", r"c & b\s+(.+?)(?=\s+\d+\s*\()", 1))
        .when(comm.rlike(PAT_CAUGHT),
             regexp_extract("commentary", r"\bc\s+(.+?)\s+b\s+", 1))
        .when(comm.rlike(PAT_LBW),
             regexp_extract("commentary", r"lbw\s+b\s+(.+?)(?=\s+\d+\s*\()", 1))
        .when(comm.rlike(PAT_STUMPED),
             regexp_extract("commentary", r"st\s+(.+?)\s+b\s+", 1))
        .when(comm.rlike(PAT_BOWLED),
             regexp_extract("commentary", r"(?:^|[^a-zA-Z])b\s+(.+?)(?=\s+\d+\s*\()", 1))
        .otherwise(lit(None)))
    .withColumn("dismissed_by", trim(regexp_replace("dismissed_by", "†", "")))
)

### Enrich with Team & Player Names + Backfill Super Overs

In [0]:
# Resolve names
df_events = (
    df_events.join(df_match_names, on="matchid", how="left")
    .withColumn("team", get_full_name("innings", "team_names"))
    .withColumn("batsman_full", get_full_name("Batsman", "player_names"))
    .withColumn("bowler_full", get_full_name("Bowler", "player_names"))
)

# ── Backfill Super Over rows ──
non_super = ~col("team").contains("Super Over")

# Batsman
bat_map = df_events.filter(non_super & col("batsman_full").isNotNull()) \
    .groupBy("Batsman").agg(first("batsman_full", ignorenulls=True).alias("_bat"))
df_events = df_events.join(bat_map, "Batsman", "left") \
    .withColumn("batsman_full", coalesce("batsman_full", "_bat")).drop("_bat")

# Bowler
bow_map = df_events.filter(non_super & col("bowler_full").isNotNull()) \
    .groupBy("Bowler").agg(first("bowler_full", ignorenulls=True).alias("_bow"))
df_events = df_events.join(bow_map, "Bowler", "left") \
    .withColumn("bowler_full", coalesce("bowler_full", "_bow")).drop("_bow")

# Super over team column + team backfill
df_events = df_events.withColumn("super_over_team",
    when(col("team").contains("Super Over"), col("team")).otherwise(lit("normal")))

team_map = df_events.filter(non_super & col("team").isNotNull()) \
    .groupBy("batsman_full").agg(first("team", ignorenulls=True).alias("_team"))
df_events = df_events.join(team_map, "batsman_full", "left") \
    .withColumn("team",
        when(col("team").contains("Super Over"), col("_team")).otherwise(col("team"))) \
    .drop("_team")

### Innings Score & Wickets Lost (Running Totals)

In [0]:
# Shared window spec for innings-level running totals
innings_window = (
    Window
    .partitionBy("matchid", "team", "super_over_team")
    .orderBy("over", "match_ball_number")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

df_events = (
    df_events
    .withColumn("innings_score", spark_sum("runs").over(innings_window))
    .withColumn("wickets_lost",
        spark_sum(
            when(col("dismissal_method") != "Not Out", 1).otherwise(0)
        ).over(innings_window))
)

# Cache events — reused for Silver output + merge + display
#df_events.cache()
df_events.createOrReplaceTempView("match_events_temp")
df_events = spark.table("match_events_temp")

### Prepare & Merge Events into Silver

In [0]:
df_events_silver = df_events.select(
    "Batchid", "matchid", "event", "ball", "over", "over_ball_number","match_ball_number",
    "runs", "innings_score", "wickets_lost", "commentary",
    col("bowler_full").alias("bowler"), col("batsman_full").alias("batsman"),
    "team", "is_super_over", "super_over_team", "runs_text", "Extras",
    "dismissal_method", "dismissed_by", "source_file", "load_timestamp",
    current_timestamp().alias("silver_load_timestamp")
)

print(f"📊 Events for Silver: {df_events_silver.count()}")

# Merge
events_existed = merge_to_silver(
    df_events_silver, SILVER_EVENTS, ["matchid", "match_ball_number", "event"]
)

# Propagate deletes
propagate_deletes(
    df_events_cdf, SILVER_EVENTS, ["matchid", "match_ball_number", "event"], events_existed
)

# Applying Transformations on Players Table

### Dedup, Prepare & Merge Players into Silver

In [0]:
df_players = dedup_dataframe(df_players_full, order_by_col="load_timestamp", keep="latest")

df_players_silver = df_players.select(
    "Batchid", "matchid", "innings", "team", "player_name",
    "batted", "batting_position", "player_type", "retired",
    "not_out", "bowled", "source_file", "load_timestamp",
    current_timestamp().alias("silver_load_timestamp")
)

print(f"📊 Players for Silver: {df_players_silver.count()}")

# Merge
players_existed = merge_to_silver(
    df_players_silver, SILVER_PLAYERS, ["matchid", "player_name", "team"]
)

# Propagate deletes
propagate_deletes(
    df_players_cdf, SILVER_PLAYERS, ["matchid", "player_name", "team"], players_existed
)

# Applying Transformations on Match Metadata Table

### Dedup, Clean Text, Parse Dates/Times, Resolve Timezones

In [0]:
# Dedup
df_metadata = dedup_dataframe(df_metadata_full, order_by_col="load_timestamp", keep="latest")

# Split toss: "Team, elected to bat first" → toss + decision
toss_parts = split(col("toss"), ",")
df_metadata = (
    df_metadata
    .withColumn("_parts", toss_parts)
    .withColumn("toss", trim(col("_parts")[0]))
    .withColumn("decision",
        when(size(col("_parts")) >= 2, trim(col("_parts")[1])).otherwise(lit(None)))
    .drop("_parts")
)

# Remove repetitive text in columns
df_metadata = dedup_column_values(df_metadata, [
    "series", "player_of_the_match", "player_of_the_series",
    "t20_debut", "t20i_debut", "umpires", "tv_umpire",
    "reserve_umpire", "match_referee"
])

# Parse match_days → match_date
df_metadata = df_metadata.withColumn("match_date",
    to_date(trim(regexp_extract("match_days", r"^(\d+\s+\w+\s+\d{4})", 1)), "d MMMM yyyy"))

# Parse hours_of_play → structured time columns
time_extractions = {
    "start_time":           r"^(\d+\.\d+)\s+start",
    "first_innings_start":  r"First Session\s+(\d+\.\d+)",
    "first_innings_end":    r"First Session\s+\d+\.\d+-(\d+\.\d+)",
    "interval_start":       r"Interval\s+(\d+\.\d+)",
    "interval_end":         r"Interval\s+\d+\.\d+-(\d+\.\d+)",
    "second_innings_start": r"Second Session\s+(\d+\.\d+)",
    "second_innings_end":   r"Second Session\s+\d+\.\d+-(\d+\.\d+)",
}

for col_name, pattern in time_extractions.items():
    df_metadata = df_metadata.withColumn(col_name,
        when(
            regexp_extract("hours_of_play_local_time", pattern, 1) == "", lit(None)
        ).otherwise(
            regexp_extract("hours_of_play_local_time", pattern, 1)
        )
    )

# Timezone: geocode grounds → timezone
timezone_map = build_ground_timezone_map(df_metadata, ground_col="ground")
df_metadata = apply_timezone_map(df_metadata, timezone_map)

# Convert local times to UTC
for local_col, utc_col in [
    ("start_time",            "match_start_utc"),
    ("first_innings_start",   "first_innings_start_utc"),
    ("first_innings_end",     "first_innings_end_utc"),
    ("second_innings_start",  "second_innings_start_utc"),
    ("second_innings_end",    "second_innings_end_utc"),
]:
    df_metadata = local_time_to_utc(df_metadata, local_col, "match_date", "local_timezone", utc_col)

# Resolve team names for first/second innings
df_metadata = (
    df_metadata.join(df_match_names, on="matchid", how="left")
    .withColumn("first_innings", get_full_name("first_innings", "team_names"))
    .withColumn("second_innings", get_full_name("second_innings", "team_names"))
)

### Prepare & Merge Metadata into Silver

In [0]:
df_metadata_silver = df_metadata.select(
    "Batchid", "matchid", "ground", "toss", "decision", "series",
    "season", "player_of_the_match", "player_of_the_series",
    "t20_debut", "t20i_debut", "umpires", "tv_umpire",
    "reserve_umpire", "match_referee", "points", "player_replacements",
    "match_number", "match_date", "match_start_utc",
    "first_innings_start_utc", "first_innings_end_utc",
    "second_innings_start_utc", "second_innings_end_utc",
    "first_innings", "second_innings",
    "has_super_over", "super_over_count", "series_result",
    "source_file", "load_timestamp",
    current_timestamp().alias("silver_load_timestamp")
)

print(f"📊 Metadata for Silver: {df_metadata_silver.count()}")

# Merge
metadata_existed = merge_to_silver(df_metadata_silver, SILVER_METADATA, ["matchid"])

# Propagate deletes
propagate_deletes(df_metadata_cdf, SILVER_METADATA, ["matchid"], metadata_existed)

## Update CDF Checkpoints

Checkpoints updated only after all Silver tables are successfully written.
If the notebook fails mid-run, next execution re-reads from the last
successful version (at-least-once + merge = idempotent).

In [0]:
for table_name, version in [
    (BRONZE_EVENTS, events_version),
    (BRONZE_PLAYERS, players_version),
    (BRONZE_METADATA, metadata_version)
]:
    if version is not None:
        update_checkpoint(table_name, version)
        print(f"  ✅ {table_name} → version {version}")

# Release cached DataFrames
#df_match_names.unpersist()
#df_events.unpersist()

print("\n🎉 Incremental Silver load via CDF complete!")

In [0]:
# Run DQ checks after ingestion completes
dq_result = dbutils.notebook.run(
    "/Workspace/Shared/cricketCommentaryScraper/DataQualityRulesSilverLayer",
    timeout_seconds=600,  # 10 min timeout,
    arguments       = {
        "catalog_name":  CATALOG
    }
)
print(f"DQ Result: {dq_result}")